## Product Review Aspect Detection: Camera Bag


### This is a Natural Language Processing based solution which can detect up to 10 aspects from online product reviews for camera bags.

This sample notebook shows you how to deploy Product Review Aspect Detection: Camera using Amazon SageMaker.

This solution identifies the various aspects from online product reviews for cameras bags. The following 7 aspects are identified: purchase, aesthetics, size, ergonomics, versatility, safety and durability  . This enables companies to easily identify which aspects are being reviewed. This information can be used to assess changes in consumer priorities and focus on trending aspects related to cameras. Companies can use this solution for providing inputs to advertising and marketing campaigns and product design.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

#### Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to Product Review Aspect Detection: Camera. If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)

#### Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Visualize output](#D.-Visualize-output)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
3. [Perform batch inference](#3.-Perform-batch-inference) 
4. [Clean-up](#4.-Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))
    

#### Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

### 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page Product Review Aspect Detection: Camera 
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

In [19]:
model_package_arn='arn:aws:sagemaker:us-east-2:XXXXXXXXXXXX:model-package/camera-bag-aspects'

### 2. Prepare dataset

In [2]:
import base64
import json 
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
from urllib.parse import urlparse
import boto3
from IPython.display import Image
from PIL import Image as ImageEdit
import urllib.request
import numpy as np

In [ ]:
role = get_execution_role()

sagemaker_session = sage.Session()

bucket=sagemaker_session.default_bucket()
bucket

### 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [5]:
model_name='camera-bag-aspect-extraction'

content_type='text/plain'

real_time_inference_instance_type='ml.m5.large'
batch_transform_inference_instance_type='ml.m5.large'

#### A. Create an endpoint

In [11]:

def predict_wrapper(endpoint, session):
    return sage.predictor.Predictor(endpoint, session,content_type)

#create a deployable model from the model package.
model = ModelPackage(role=role,
                    model_package_arn=model_package_arn,
                    sagemaker_session=sagemaker_session,
                    predictor_cls=predict_wrapper)

#Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

----!

Once endpoint has been created, you would be able to perform real-time inference.

#### B. Create input payload

In [12]:
# Provide the text file name with its location
file_name = '/input.txt'

<Add code snippet that shows the payload contents>

#### C. Perform real-time inference

In [13]:
!aws sagemaker-runtime invoke-endpoint \
    --endpoint-name $model_name \
    --body fileb://$file_name \
    --content-type $content_type \
    --region $sagemaker_session.boto_region_name \
    output.txt

{
    "ContentType": "application/json",
    "InvokedProductionVariant": "AllTraffic"
}


#### D. Visualize output

In [14]:
import json
with open('output.txt', 'r') as f:
    output = json.load(f)
print(json.dumps(output, indent = 1))

{
 "review": "I have had and used (borderline abused!) my canon camera bag for about 10 years and can't recommend it enough. It is has taken a beating but still looks great and has held up really well. The bag is sturdy and protective. It has perfect amount of space for my Canon 60D, with enough room to cary an additional small micro-lens. (When I had a smaller DLSR, the bag perfectly held the camera body and 2 larger lenses). I recently repurchased this camera bag for my father.",
 "topics": [
  {
   "aspect": {
    "versatality": 0.4850679692606684
   },
   "sentence": "borderline abuse"
  },
  {
   "aspect": {
    "size": 0.3065129392699415
   },
   "sentence": "canon camera bag year recommend"
  },
  {
   "aspect": {
    "asthetics": 0.5131470989408985
   },
   "sentence": "take beating look great hold"
  },
  {
   "aspect": {
    "asthetics": 0.3703455002942877,
    "safety": 0.3419004271639299
   },
   "sentence": "bag sturdy protective"
  },
  {
   "aspect": {
    "size": 0.3912

#### E. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [15]:
predictor=sage.predictor.Predictor(model_name, sagemaker_session,content_type)
predictor.delete_endpoint(delete_endpoint_config=True)

### 3. Perform batch inference

In this section, you will perform batch inference using multiple input payloads together. If you are not familiar with batch transform, and want to learn more, see these links:
1. [How it works](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-batch-transform.html)
2. [How to run a batch transform job](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html)

In [16]:
#upload the batch-transform job input files to S3 and provide the location here
transform_input_folder = "input"
transform_input = sagemaker_session.upload_data(transform_input_folder, key_prefix=model_name) 
print("Transform input uploaded to " + transform_input)

Transform input uploaded to s3://sagemaker-us-east-2-786796469737/camera-bag-aspect-extraction


In [ ]:

#Run the batch-transform job
transformer = model.transformer(1, batch_transform_inference_instance_type)
transformer.transform(transform_input, content_type=content_type)
transformer.wait()

........................./usr/local/lib/python3.6/dist-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.20.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
 * Serving Flask app 'serve' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on all addresses.
 * Running on http://169.254.255.131:8080/ (Press CTRL+C to quit)
169.254.255.130 - - [19/Nov/2021 07:48:43] "GET /ping HTTP/1.1" 200 -
169.254.255.130 - - [19/Nov/2021 07:48:43] "#033GET /execution-parameters HTTP/1.1#033" 404 -
need at least one array to concatenate
169.254.255.130 - - [19/Nov/2021 07:48:43] "#033#033POST /invocations HTTP/1.1#033" 500 -
Error on request:
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/werkzeug/serving.py", line 319, in run_wsgi
    execute(self.server.app)
  File "/usr/local/lib/pytho

In [ ]:
import os
s3_conn = boto3.client("s3")
with open('output2.txt', 'wb') as f:
    s3_conn.download_fileobj(bucket, os.path.basename(transformer.output_path)+'/sample.txt.out', f)
    print("Output file loaded from bucket")

In [ ]:
with open('output2.txt', 'r') as f:
    output = json.load(f)
print(json.dumps(output, indent = 1))

### 4. Clean-up

#### A. Delete the model

In [ ]:
model.delete_model()

#### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

